# HW-9 Cluster
Wyatt Blair

SID: 10420296

4/16/24


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.metrics import confusion_matrix, f1_score
import seaborn as sns

tqdm.pandas()

In [2]:
dataset = pd.read_csv('../data/wisc_bc_ContinuousVar.csv', index_col='id')
dataset['diagnosis'].replace(["M", "B"], [1, 0], inplace=True)
dataset.dropna(inplace=True)

dataset

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
id,,,,,,,,,,,,,,,,,,,,,
842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926424,1,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
926682,1,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
926954,1,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820


In [3]:
def min_max_normalization(series):

    minimum, maximum = series.min(), series.max()
    normalized_data = (series - minimum) / (maximum - minimum)

    return normalized_data

def z_score_standardization(series):

    mean, std = series.mean(), series.std()
    normalized_data = (series - mean) / std

    return normalized_data

In [4]:
target = 'diagnosis'
features = dataset.columns.to_list(); features.remove(target)

In [5]:
normalized_dataset = dataset.copy()
normalized_dataset[features] = min_max_normalization(normalized_dataset[features])

normalized_dataset

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
id,,,,,,,,,,,,,,,,,,,,,
842302,1,0.521037,0.022658,0.545989,0.363733,0.593753,0.792037,0.703140,0.731113,0.686364,...,0.620776,0.141525,0.668310,0.450698,0.601136,0.619292,0.568610,0.912027,0.598462,0.418864
842517,1,0.643144,0.272574,0.615783,0.501591,0.289880,0.181768,0.203608,0.348757,0.379798,...,0.606901,0.303571,0.539818,0.435214,0.347553,0.154563,0.192971,0.639175,0.233590,0.222878
84300903,1,0.601496,0.390260,0.595743,0.449417,0.514309,0.431017,0.462512,0.635686,0.509596,...,0.556386,0.360075,0.508442,0.374508,0.483590,0.385375,0.359744,0.835052,0.403706,0.213433
84348301,1,0.210090,0.360839,0.233501,0.102906,0.811321,0.811361,0.565604,0.522863,0.776263,...,0.248310,0.385928,0.241347,0.094008,0.915472,0.814012,0.548642,0.884880,1.000000,0.773711
84358402,1,0.629893,0.156578,0.630986,0.489290,0.430351,0.347893,0.463918,0.518390,0.378283,...,0.519744,0.123934,0.506948,0.341575,0.437364,0.172415,0.319489,0.558419,0.157500,0.142595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
926424,1,0.690000,0.428813,0.678668,0.566490,0.526948,0.296055,0.571462,0.690358,0.336364,...,0.623266,0.383262,0.576174,0.452664,0.461137,0.178527,0.328035,0.761512,0.097575,0.105667
926682,1,0.622320,0.626987,0.604036,0.474019,0.407782,0.257714,0.337395,0.486630,0.349495,...,0.560655,0.699094,0.520892,0.379915,0.300007,0.159997,0.256789,0.559450,0.198502,0.074315
926954,1,0.455251,0.621238,0.445788,0.303118,0.288165,0.254340,0.216753,0.263519,0.267677,...,0.393099,0.589019,0.379949,0.230731,0.282177,0.273705,0.271805,0.487285,0.128721,0.151909


In [6]:
train = normalized_dataset.sample(frac=0.8)
test = normalized_dataset.drop(index=train.index)

___
## 9.1

Using hclust, categorize the “wisc_bc_ContinuousVar.csv” data into two (2) clusters based on. All the features except the diagnosis column. Tabulate the clustered rows against the “diagnosis” column. (Remove the rows with missing values first)


#### Trying it from scratch

In [7]:
def minkowski_distance(x, y, p):

    diff = x - y
    power = diff ** p

    total = power.sum(axis=len(x.shape)-1)
    distance = total ** (1/p)

    return distance

def euclidean_distance(x, y):
    return minkowski_distance(x, y, p=2)

def manhattan_distance(x, y):

    diff = x - y
    absolute = abs(diff)
    distance = absolute.sum(axis=1)

    return distance

In [8]:
class HierarchicalClustering:

    def __init__(self, X, distance_func=euclidean_distance, final_num_clusters=2, verbose=True):

        self.X = X
        self.N, self.M = self.X.shape

        self.distance_func = distance_func
        self.final_num_clusters = final_num_clusters
        self.verbose = verbose

        self.clusters = pd.Series(X.index, index=X.index, name='cluster_id')
        self.distance_mat = self.calc_distance_mat()

        self.merge_log = []

    def calc_distance_mat(self):
        
        cluster_centers = self.get_cluster_centers()
        N = len(cluster_centers)
        distance_mat = pd.DataFrame(np.zeros((N, N)), index=cluster_centers.keys(), columns=cluster_centers.keys())

        for i_ind, i in enumerate(distance_mat.index):
            for j_ind, j in enumerate(distance_mat.columns):
                
                # don't want to consider self-comparisons during minimization later
                if i == j: 
                    distance_mat[i][j] = np.inf
                if i_ind >= j_ind:
                    continue

                x, y = cluster_centers[i].values, cluster_centers[j].values
                distance = self.distance_func(x, y)

                # fill in other half off matrix at the same time
                distance_mat[i][j] = distance
                distance_mat[j][i] = distance

        return distance_mat
    
    def get_cluster_centers(self):

        cluster_centers = {}
        for cluster_id in self.clusters.unique():
            
            point_ids_in_cluster = self.clusters[self.clusters == cluster_id].index
            points_in_cluster = self.X[self.X.index.isin(point_ids_in_cluster)]
            
            center = points_in_cluster.mean(axis=0)
            cluster_centers.update({cluster_id: center})
        
        return cluster_centers

    def merge_clusters(self):

        initial_num_clusters = len(self.clusters.unique())

        # Identify minimum pair in distance matrix
        absorbing_cluster_ind, absorbed_cluster_ind = np.unravel_index(
            np.argmin(self.distance_mat), 
            shape=self.distance_mat.shape
        )

        absorbing_cluster_id = self.distance_mat.index[absorbing_cluster_ind]
        absorbed_cluster_id = self.distance_mat.columns.to_list()[absorbed_cluster_ind]
        self.merge_log.append(f'{absorbed_cluster_id} absorbed by {absorbing_cluster_id}')

        # perform cluster merge
        self.clusters[self.clusters == absorbed_cluster_id] = self.clusters[absorbing_cluster_id]

        # update distance matrix
        self.distance_mat = self.calc_distance_mat()

        # sanity check
        final_num_clusters = len(self.clusters.unique())
        try:
            assert (final_num_clusters - initial_num_clusters == -1)
        except AssertionError:
            import pdb; pdb.set_trace()

    def fit(self):

        train_loop = range(self.N)
        if self.verbose:
            train_loop = tqdm(train_loop, desc='Clustering', total=len(train_loop))

        for cluster_merge_ind in train_loop:
            self.merge_clusters()
    
    

In [9]:
hclust = HierarchicalClustering(
    train[features],
    distance_func=euclidean_distance,
    final_num_clusters=2,
    verbose=True
)

In [10]:
hclust.fit()

Clustering:   0%|          | 0/455 [00:00<?, ?it/s]

--Return--
None
> /var/folders/h0/mkxp1q6d01g2qycl48fndr680000gn/T/ipykernel_6268/4261129896.py(79)merge_clusters()
     77             assert (final_num_clusters - initial_num_clusters == -1)
     78         except AssertionError:
---> 79             import pdb; pdb.set_trace()
     80 
     81     def fit(self):



In [11]:
hclust.clusters.unique()

array([8711561])

In [12]:
hclust.merge_log

['9010598 absorbed by 8610908',
 '9112594 absorbed by 9112367',
 '922576 absorbed by 861648',
 '844359 absorbed by 894326',
 '8610175 absorbed by 868682',
 '8912521 absorbed by 857810',
 '8910748 absorbed by 90317302',
 '8610908 absorbed by 862009',
 '8810528 absorbed by 912193',
 '91789 absorbed by 907367',
 '891716 absorbed by 871122',
 '875878 absorbed by 911202',
 '914580 absorbed by 9011495',
 '909410 absorbed by 915664',
 '9013005 absorbed by 911202',
 '911202 absorbed by 9010877',
 '905189 absorbed by 867387',
 '9011495 absorbed by 911320501',
 '88518501 absorbed by 857374',
 '89511501 absorbed by 871122',
 '901028 absorbed by 86561',
 '904357 absorbed by 8711003',
 '8711003 absorbed by 868682',
 '862009 absorbed by 868682',
 '911320501 absorbed by 868682',
 '912193 absorbed by 868682',
 '863270 absorbed by 868682',
 '88350402 absorbed by 908469',
 '922577 absorbed by 88147101',
 '895299 absorbed by 90769601',
 '901041 absorbed by 861648',
 '862485 absorbed by 906024',
 '915664 

#### Pre-Made implementation

In [21]:
import plotly.figure_factory as ff

fig = ff.create_dendrogram(normalized_dataset[features])
fig.update_layout(width=800, height=500)
fig.show()

  Obtaining dependency information for plotly from https://files.pythonhosted.org/packages/00/4e/6258fc3b26f1f7abd1b2e75b1e9e4f12f13584136e2e1549f995ff4c6b7b/plotly-5.20.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 77.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


___
## 9.2

Using k-means, categorize the “wisc_bc_ContinuousVar.csv” data into two (2) clusters based on. All the features except the diagnosis column. Tabulate the clustered rows against the “diagnosis” column. (Remove the rows with missing values first)

### K-Means

#### Assignment Step
$S_i^{(t)} = \{x_p : ||x_p - m_i^{(t)}||^2 \leq ||x_p - m_j^{(t)}||^2 ; \forall{j}, 1\leq j \leq k \}$ 

#### Update Step
$m_i^{(t+1)} = \frac{1}{|S_i^{(t)}} \sum_{x_j \in S_i^{(t)}}{x_j}$

In [13]:
class KMeansClustering:

    def __init__(self, X: pd.DataFrame, k: int):

        self.X = X
        self.N, self.M = self.X.shape
        self.k = k

        self.random_start_points = self.X.sample(n=self.k)
        self.centroids = {k_ind: centroid for k_ind, centroid in enumerate(self.random_start_points.values)}
        self.centroids = pd.DataFrame().from_dict(self.centroids, orient='index', columns=self.X.columns)

    def assignment_step(self):
        
        self.distance_mat = pd.DataFrame(np.zeros((self.N, self.k)), index=self.X.index, columns=self.centroids.index)
        for centroid_ind, centroid in self.centroids.iterrows():
            
            tiled_centroid = np.repeat(centroid.values[:, np.newaxis], repeats=self.N, axis=1).T
            distances = euclidean_distance(tiled_centroid, self.X)
            self.distance_mat[centroid_ind] = distances
        
        self.clusters = self.distance_mat.idxmin(axis=1)
        return self.clusters

    def update_step(self):
        
        for k_ind in self.centroids.index:
            
            cluster_mask = self.clusters.values == k_ind
            points_in_cluster_inds = self.clusters[cluster_mask].index
            points_in_cluster = self.X.loc[points_in_cluster_inds]
            
            new_centroid = points_in_cluster.sum(axis=0) / len(points_in_cluster.index)
            self.centroids.loc[k_ind] = new_centroid

    def fit(self):
        
        
        # initialize vars
        init_clusters = self.assignment_step()
        self.update_step()
        fin_clusters = self.assignment_step()

        # keep going until the clusters are stable
        while (init_clusters != fin_clusters).any():
            
            init_clusters = fin_clusters
            self.update_step()
            fin_clusters = self.assignment_step()



In [14]:
k_means = KMeansClustering(
    X=normalized_dataset[features],
    k=2,
)

In [15]:
k_means.fit()

In [16]:
k_means.clusters.value_counts()

1    380
0    189
Name: count, dtype: int64

In [17]:
normalized_dataset[target].value_counts()

diagnosis
0    357
1    212
Name: count, dtype: int64

In [23]:
clusters_labels_swapped = k_means.clusters.copy()
clusters_labels_swapped.replace({1: 0, 0: 1}, inplace=True)

In [18]:
normalized_dataset['prediction'] = k_means.clusters
normalized_dataset['correct'] = normalized_dataset['prediction'] == normalized_dataset[target]

acc = normalized_dataset['correct'].value_counts()[True] / len(normalized_dataset.index)
print('Accuracy %1.3f%%' % (acc*100))

Accuracy 7.206%


In [24]:
normalized_dataset['prediction-swapped'] = clusters_labels_swapped
normalized_dataset['correct'] = normalized_dataset['prediction-swapped'] == normalized_dataset[target]

acc = normalized_dataset['correct'].value_counts()[True] / len(normalized_dataset.index)
print('Accuracy %1.3f%%' % (acc*100))

Accuracy 92.794%
